In [4]:
import numpy as np
from astropy import units as u
from astropy import constants as const



In [29]:
#calculate the Knu using adams code from https://github.com/keflavich/dust_emissivity

"""
===============
Dust emissivity
===============
"""
import numpy as np
import os
import requests
from astropy.table import Table
from astropy import constants
from astropy import units as u
from numpy import exp,log
from scipy.integrate import quad

def kappa(nu, nu0=271.1*u.GHz, kappa0=0.0114*u.cm**2*u.g**-1, beta=1.75):
    """
    Compute the opacity $\kappa$ given a reference frequency (or wavelength)
    and a power law governing the opacity as a fuction of frequency:

    $$ \kappa = \kappa_0 \left(\\frac{\\nu}{\\nu_0}\\right)^{\\beta} $$

    The default kappa=0.0114 at 271.1 GHz comes from extrapolating the
    Ossenkopf & Henning 1994 opacities for the thin-ice-mantle, 10^6 year model
    anchored at 1.0 mm with an assumed beta of 1.75.

    Parameters
    ----------
    nu: astropy.Quantity [u.spectral() equivalent]
        The frequency at which to evaluate kappa
    nu0: astropy.Quantity [u.spectral() equivalent]
        The reference frequency at which $\kappa$ is defined
    kappa0: astropy.Quantity [cm^2/g]
        The dust opacity per gram of H2 along the line of sight.  Because of
        the H2 conversion, this factor implicitly includes a dust to gas ratio
        (usually assumed 100)
    beta: float
        The power-law index governing kappa as a function of nu
    """
    return (kappa0*(nu.to(u.GHz,u.spectral())/nu0.to(u.GHz,u.spectral()))**(beta)).to(u.cm**2/u.g)





In [32]:
#http://articles.adsabs.harvard.edu/pdf/1994A%26A...291..943O

#    mm           Ghz         Knu
#    1.3           230GHz    0.023cm^2/g
#    3             90



nu = (90 * u.GHz).cgs
#Knu= 0.023 * u.cm**2/u.g 
Knu = kappa(nu)

dist = (131* u.pc).cgs

Rdisk = ([75,100,150]* u.au).cgs
Tdisk = (20* u.K).cgs
Mdisk = (0.009*u.solMass).cgs #MMEN



Ncol = Mdisk/(np.pi*(Rdisk)**2)

tau = Knu * Ncol

BBmod = (2*(const.h).cgs*nu**3)/((const.c).cgs**2)* (np.e**(((const.h).cgs*nu)/((const.k_B).cgs*Tdisk))-1)**-1 * (1-np.e**-tau)

Snu = (BBmod * (Rdisk**2/(4*dist**2))*4*np.pi).to(u.mJy)

print(Snu)




###signal to noise time calc
'''
SNR = 10      #desired snr
noise = 0.43* u.mJy #calculated from SMA calculator, mJy
t0 = 6.52*u.hr    #time in hrs for an SMA track
'''


#https://www.gb.nrao.edu/mustang/
#4'x4' fov with mustang2

SNR = 10      #desired snr
noise = 0.056 * u.mJy 
#noise = 56* u.uJy
t0 = 1*u.hr

time= t0*(SNR/(Snu/noise))**2

print(f'estimated on source time for SNR of {SNR} = {time}')

[0.80533899 0.80665827 0.80760238] mJy
estimated on source time for SNR of 10 = [0.48352463 0.48194433 0.48081818] h
